In [1]:
import pandas as pd  #  type: ignore
import numpy as np

# Cargar los datos
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    compression="zip",
)
#train_data.head()

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    compression="zip",
)
#test_data.head()

# Función para limpiar los datasets
def clean_dataset(data):
    # Renombrar la columna 'default payment next month' a 'default'
    data = data.rename(columns={"default payment next month": "default"})
    # Eliminar la columna 'ID'
    if "ID" in data.columns:
        data = data.drop(columns=["ID"])
    
    # Convertir valores no válidos a NaN
    data['EDUCATION'] = data['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
    data['MARRIAGE'] = data['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)

    # Agrupar valores mayores a 4 en la columna 'EDUCATION' como "others" (valor 4)
    data['EDUCATION'] = data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
    
    # Validación explícita: Imprimir valores únicos
    print("Valores únicos en 'EDUCATION' después de agrupar mayores a 4:")
    print(data['EDUCATION'].unique())
    
    # Eliminar registros con valores faltantes
    data = data.dropna()

    # Validación final: Asegurarnos de que no haya valores mayores a 4
    if (data['EDUCATION'] > 4).any():
        print("Error: Existen valores mayores a 4 en 'EDUCATION'")
    else:
        print("Limpieza de 'EDUCATION' completada correctamente.")
    
    return data

# Limpiar los datasets
dataset_train = clean_dataset(dataset_train)
dataset_test = clean_dataset(dataset_test)


Valores únicos en 'EDUCATION' después de agrupar mayores a 4:
[ 3.  2.  1.  4. nan]
Limpieza de 'EDUCATION' completada correctamente.
Valores únicos en 'EDUCATION' después de agrupar mayores a 4:
[ 2.  3.  1.  4. nan]
Limpieza de 'EDUCATION' completada correctamente.


In [2]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
y_train=dataset_train.copy()
y_train=y_train.pop("default")

x_train = dataset_train.copy()
x_train.pop("default")

y_test=dataset_test.copy()
y_test=y_test.pop("default")

x_test = dataset_test.copy()
x_test.pop("default")

0       1
1       0
2       0
3       0
4       0
       ..
8995    0
8996    0
8997    0
8998    1
8999    1
Name: default, Length: 8979, dtype: int64

In [3]:

# - Escala las demas variables al intervalo [0, 1].
# - Selecciona las K mejores caracteristicas.
# - Ajusta un modelo de regresion logistica.

# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

variables_categoricas = ["SEX", "EDUCATION", "MARRIAGE"] 

# Crear el preprocesador que aplica el transformador a las columnas categóricas
# - Escala las demas variables al intervalo [0, 1].
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown="ignore"), ['SEX', 'EDUCATION', 'MARRIAGE']),
        ('num', MinMaxScaler(), list(set(x_test.columns) - set(variables_categoricas)))
    ],
    remainder='passthrough'
)

# - Selecciona las K mejores caracteristicas.
from sklearn.feature_selection import SelectKBest,  f_classif

# Crear el pipeline que incluye el preprocesador, el scaler, el selector y el modelo de regresion logistica
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(score_func=f_classif, k=10)),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

#Entrenar el pipeline
pipeline.fit(x_train, y_train)

print("\nPipeline de clasificación ajustado exitosamente.")


Pipeline de clasificación ajustado exitosamente.


In [4]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
#
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score

# Definir los hiperparametros a optimizar
param_grid = {
    'selector__k': range(1, len(x_train.columns) + 1),
    "classifier__C": [0.01, 0.1, 1, 10, 80],  # Regularización de la regresión logística
    "classifier__penalty": ["l2"],  # Tipo de penalización
    "classifier__solver": ["lbfgs", "liblinear", "saga"],  # Solvers compatibles con L2
}



# Realizar la búsqueda de hiperparametros
best_model = GridSearchCV(
    pipeline, param_grid, cv=10, scoring='balanced_accuracy', n_jobs=-1)


# Ajustar el modelo
best_model.fit(x_train, y_train)

# Mostrar los mejores parámetros encontrados y la puntuación
print("\nMejores parámetros encontrados:")
print(best_model.best_params_)
print(f"Mejor precisión balanceada (validación cruzada): {best_model.best_score_:.4f}")

# Actualizar el pipeline con los mejores parámetros encontrados
best_model_1 = best_model.best_estimator_


Mejores parámetros encontrados:
{'classifier__C': 1, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs', 'selector__k': 1}
Mejor precisión balanceada (validación cruzada): 0.6393


In [5]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.
import os
import joblib
import gzip
import pickle

# Guardar el objeto GridSearchCV completo
ruta_modelo = '../files/models/model.pkl.gz'
os.makedirs(os.path.dirname(ruta_modelo), exist_ok=True)
with gzip.open(ruta_modelo, 'wb') as f:
     pickle.dump(best_model, f)

In [6]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'type': 'metrics', 'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#

import json

# Calcular las metricas
metrics = { 'type': "metrics",
    'dataset': 'train',
    'precision': precision_score(y_train, best_model_1.predict(x_train)),
    'balanced_accuracy': balanced_accuracy_score(y_train, best_model_1.predict(x_train)),
    'recall': recall_score(y_train, best_model_1.predict(x_train)),
    'f1_score': f1_score(y_train, best_model_1.predict(x_train)),
}

metrics_test = { 'type': "metrics",
    'dataset': 'test',
    'precision': precision_score(y_test, best_model_1.predict(x_test)),
    'balanced_accuracy': balanced_accuracy_score(y_test, best_model_1.predict(x_test)),
    'recall': recall_score(y_test, best_model_1.predict(x_test)),
    'f1_score': f1_score(y_test, best_model_1.predict(x_test)),}

# Guardar las métricas en formato JSON
ruta= "../files/output"
ruta_json= os.path.join(ruta, "metrics.json")

# Crear el directorio
os.makedirs(ruta, exist_ok=True)

with open(ruta_json, 'w') as f:
    json.dump(metrics, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(metrics_test, f, ensure_ascii=False)  
    f.write('\n')

In [7]:
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

from sklearn.metrics import confusion_matrix

# Calcular las matrices de confusión
cm_train = confusion_matrix(y_train, best_model_1.predict(x_train))
cm_test = confusion_matrix(y_test, best_model_1.predict(x_test))

cm_train_dict = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {'predicted_0': int(cm_train[0, 0]), 'predicted_1': int(cm_train[0, 1])},
    'true_1': {'predicted_0': int(cm_train[1, 0]), 'predicted_1': int(cm_train[1, 1])}
}

cm_test_dict = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {'predicted_0': int(cm_test[0, 0]), 'predicted_1': int(cm_test[0, 1])},
    'true_1': {'predicted_0': int(cm_test[1, 0]), 'predicted_1': int(cm_test[1, 1])}
}


#guardar en json
with open(ruta_json, 'a', encoding='utf-8') as f:
    json.dump(cm_train_dict, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(cm_test_dict, f, ensure_ascii=False)  
    f.write('\n')

In [8]:
with gzip.open("../files/models/model.pkl.gz", "rb") as file:
        model = pickle.load(file)
str(type(model))


"<class 'sklearn.model_selection._search.GridSearchCV'>"